# **Importamos las dependencias**

In [4]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

from google.cloud import storage
from typing import List

import math

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [5]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

In [6]:
from vertexai.generative_models import GenerationConfig
from vertexai.generative_models import GenerationResponse

# **Configuración de las credenciales**

In [16]:
# --- Carga las Variables de Entorno ---
load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")                        # ID del proyecto de Google Cloud
LOCATION = os.getenv("LOCATION")                            # Región donde se encuentran los recursos de Vertex AI
INDEX_ID = os.getenv("INDEX_ID")                            # ID del índice de búsqueda vectorial en Vertex AI
ENDPOINT_ID = os.getenv("ENDPOINT_ID")                      # ID del endpoint de búsqueda vectorial en Vertex AI
BUCKET_NAME = os.getenv("BUCKET_NAME")                      # Nombre del bucket de Google Cloud Storage donde se almacenan los videos
VIDEO_FOLDER_PATH = os.getenv("VIDEO_FOLDER_PATH")          # Ruta del folder dentro del bucket donde se encuentran los videos

# Verificamos que las variebles de entorno esten bien
# print(f"Proyecto: {PROJECT_ID}, Ubicación: {LOCATION}, Índice: {INDEX_ID}, Endpoint: {ENDPOINT_ID}, Bucket: {BUCKET_NAME}, Ruta de Videos: {VIDEO_FOLDER_PATH}")

python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 19
python-dotenv could not parse statement starting at line 28
python-dotenv could not parse statement starting at line 29
python-dotenv could not parse statement starting at line 30
python-dotenv could not parse statement starting at line 32
python-dotenv could not parse statement starting at line 33


In [17]:
# Iniciamos Vertex AI con el proyecto y la ubicación especificados.
init(project = PROJECT_ID, location = LOCATION)

In [18]:
# --- Inicialización del LLM ---
# model = GenerativeModel('gemini-2.5-flash') # Debo probar con varios modelos, para ver cual tiene mejor rendimiento.

# **Funciones**

In [19]:
def obtener_nombres_de_archivos(nombre_bucket: str, nombre_carpeta: str) -> List[str]:
    """
    Obtiene una lista con los nombres de los archivos de una "carpeta" en un bucket.

    Args:
        nombre_bucket: El nombre del bucket de Cloud Storage.
        nombre_carpeta: La ruta de la carpeta (prefijo) dentro del bucket.

    Returns:
        Una lista de strings con los nombres de los archivos.
        Retorna una lista vacía si la carpeta no existe o hay un error.
    """
    try:
        cliente_storage = storage.Client()
        nombres_archivos = []

        # Aseguramos que el prefijo termine con '/' para que actúe como una carpeta
        if nombre_carpeta and not nombre_carpeta.endswith('/'):
            nombre_carpeta += '/'

        # Usamos el parámetro 'prefix' para filtrar por la carpeta
        blobs = cliente_storage.list_blobs(nombre_bucket, prefix=nombre_carpeta)

        for blob in blobs:
            # Obtiene el nombre del archivo relativo a la carpeta que estamos viendo
            nombre_relativo = blob.name.replace(nombre_carpeta, '', 1)
            if nombre_relativo: # Evita añadir una entrada vacía si la carpeta misma aparece
                nombres_archivos.append(nombre_relativo)
        
        return nombres_archivos

    except Exception as e:
        print(f"Ocurrió un error al acceder al bucket: {e}")
        return []


In [35]:
def reporteGemini(uri, marca_a_buscar, temperatura=0.0):

    """
    Analiza el video y te devuelve la data de las apariciones de la marca
    """

    # --- Configuración del modelo ---

    # Construimos la uri completa (solo tenemos el nombre del archivo)
    uri = 'gs://vboxiooof/Videos/Videos_Segmentados/'+uri


    # Preparamos al video en el formato que puede consumir Gemini
    video = Part.from_uri(uri=uri, mime_type='video/mkv')

    # Configuramos otros parámetros del modelo de Gemini
    generation_config = {'temperature': temperatura} 

    # Definimos el system prompt
    # Reemplaza tu system_instruction con esta nueva versión más precisa

    # Reemplaza tu system_instruction con esta versión HÍBRIDA

    system_instruction = """
Eres un analista de medios y publicidad de élite. Tu análisis debe ser contextual y preciso, siguiendo un proceso de decisión en dos etapas.

---
**ETAPA 1: ANÁLISIS DEL CONTEXTO GENERAL DEL CLIP**
---
Primero, evalúa la naturaleza general del video completo.
- ¿Es una **Transmisión en Vivo** del partido, mostrando el juego, a los jugadores en la cancha o a los fans en las gradas, sobre la cual se superponen gráficos?
- ¿O es un **Anuncio Producido**, que interrumpe el juego para contar una historia o mensaje publicitario con edición cinematográfica, música y actores, aunque muestre escenas de fútbol?

**Esta primera evaluación es la más importante.**

---
**ETAPA 2: CLASIFICACIÓN BASADA EN EL CONTEXTO**
---
Una vez determinado el contexto, aplica la siguiente regla:

1.  **Si el contexto es un 'Anuncio Producido':**
    La respuesta DEBE contener una única entrada para la marca. Usa la categoría **'Anuncio'** para la duración completa del clip. **NO reportes las apariciones individuales** de la marca dentro del propio comercial (en vallas, playeras, etc.).

2.  **Si el contexto es una 'Transmisión en Vivo':**
    Entonces, y solo entonces, procede a identificar y reportar cada aparición individual de la marca, eligiendo la categoría más apropiada de la siguiente lista:
    - **'Virtual'**: Gráfico generado por computadora superpuesto al campo mientras el juego continúa viéndose de fondo (ej. la botella de Yakult flotando).
    - **'Valla'**: La marca en las vallas LED físicas que rodean el campo.
    - **'Spot'**: Un gráfico que se sobrepone a la imagen sin estar integrado en el campo.
    - **'Banner'**: Un gráfico en los bordes de la pantalla.
    - **'Marcador'**: La marca junto al marcador de resultado.
    - **'Playera'**: El logo en la camiseta.
    - **'Menciones verbales'**: En comentarios o anuncios

### Formato de Salida Obligatorio
- Tu respuesta debe ser únicamente un objeto JSON con la clave "apariciones".
- Si no hay apariciones, devuelve: `{"apariciones": []}`.
- No incluyas explicaciones. Solo el JSON.

**Ejemplo de la estructura exacta que debes seguir:**
```json
{
  "apariciones": [
    {
      "marca": "Coca",
      "categoria": "Valla",
      "inicio": "00:11",
      "fin": "00:15"
    }
  ]
}

### Notas importantes
Los clips duran en total 16 segundos, entonces no puedes decir algo del estilo:
```json
{
  "apariciones": [
    {
      "marca": "Coca",
      "categoria": "Valla",
      "inicio": "00:19",
      "fin": "00:22"
    }
  ]
}

Si detectas más de una aparición en un solo clip, ten mucho cuidado con los tiempos

"""

    # Definimos la marca
    # marca_a_buscar = 'Caliente.mx'

    
    # Definimos el prompt
    prompt = "Busca en el video adjunto todas las apariciones (si es que existen) de la siguiente marca: "

    model = GenerativeModel('gemini-2.5-pro', system_instruction=system_instruction)


    # --- Hacemos la petición ---

    contenido = [video, prompt, marca_a_buscar]

    # Le pasamos a Gemini el contenido
    respuesta = model.generate_content(

        contents=contenido,
        generation_config=generation_config
        
    )


    # --- Retornamos la respuesta ---
    return respuesta.text

In [22]:
def reporteAlain(uri, marca_usuario):
    """
    Analiza el video y te devuelve la data de las apariciones de la marca.
    """
    # --- Configuración del modelo ---

    uri = f'gs://vboxiooof/Videos/Videos_Segmentados/{uri}'
    video = Part.from_uri(uri=uri, mime_type='video/mkv')
    generation_config = {'temperature': 0}

    # 1. Mueve tus instrucciones a una variable 'system_instruction'
    #    (He ajustado el f-string para que siga funcionando)
    system_instruction = f"""
        Eres un especialista en análisis de contenido deportivo con capacidad avanzada para detectar 
        marcas comerciales en videos. Sigue estrictamente estas instrucciones:

        1. BUSCAR ESPECÍFICAMENTE y ÚNICAMENTE la marca: {marca_usuario}
        2. Analizar TODO el video frame por frame si es necesario
        3. Considerar estas formas de aparición:
           - Menciones verbales (en comentarios o anuncios)
           - Tapetes al lado de porterías
           - Vallas electronicas
           - Marcador (que se encuentra al lado del marcador)
           - Banner (anuncios rectangulares, no cubren toda la pantalla, tienden a aparecer en la parte inferior o superior o a los lados de la pantalla)
           - Anuncios (comerciales de medio tiempo)
           - Virtual (tipo de anuncio que deja ver la cancha por detras, pueden ser animaciones)

        4. Para CADA detección proveer:
           - TIEMPO INICIO: (formato HH:MM:SS)
           - TIPO DE MEDIO: (seleccionar de lista anterior)
           - TAMAÑO: (pequeño <10% pantalla, mediano 10-30%, grande >30%)
           - UBICACIÓN: (esquina superior derecha, centro, etc.)
           - MENCION VERBAL: (si aplica, con tiempo exacto)

        5. Si NO se detecta la marca:
           - Indicar "Marca no encontrada después de análisis completo"
           - No dar detalles adicionales

        6. Formato de respuesta obligatorio:
        [INICIO ANÁLISIS]
        Video: {uri}
        Marca buscada: {marca_usuario}
        ---
        Detecciones:
        [Detección 1]
        Tiempo: HH:MM:SS - HH:MM:SS
        Tipo: [TIPO DE MEDIO]
        Tamaño: [TAMAÑO]
        Ubicación: [UBICACIÓN]
        Contexto: [DESCRIPCIÓN]
        Mencionada: [SI/NO] (HH:MM:SS si aplica)
        ---
        [Detección N]...
        ---
        Resumen: Total de apariciones: X
        [FIN ANÁLISIS]

        IMPORTANTE: Si el video es de baja calidad o la marca aparece muy breve, 
        igualmente reportarla si es identificable.
    """
    
    # 2. Inicializa el modelo PASANDO el system_instruction
    model = GenerativeModel('gemini-2.5-pro', system_instruction=system_instruction)

    # 3. Usa un prompt simple para la petición. La marca y las instrucciones ya están en el sistema.
    prompt = f"Analiza el video adjunto y encuentra la marca especificada en tus instrucciones."

    # --- Hacemos la petición ---
    
    # El contenido ahora solo necesita el video y el prompt sencillo
    contenido = [video, prompt]

    respuesta = model.generate_content(
        contents=contenido,
        generation_config=generation_config
    )

    # --- Retornamos la respuesta ---
    return respuesta.text

In [27]:
print(reporteGemini('mexicosta_segment_000.mkv', 'Caliente'))

```json
{
  "apariciones": [
    {
      "marca": "Caliente",
      "categoria": "Valla",
      "inicio": "00:10",
      "fin": "00:14"
    }
  ]
}
```


In [28]:
print(reporteAlain('mexicosta_segment_000.mkv', 'Caliente'))

[INICIO ANÁLISIS]
Video: gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_000.mkv
Marca buscada: Caliente
---
Detecciones:
[Detección 1]
Tiempo: 00:00:10 - 00:00:15
Tipo: Valla electronica
Tamaño: pequeño
Ubicación: esquina inferior derecha (en el fondo de la ventana del comentarista)
Contexto: El logo de "Caliente.mx" es visible en una valla electrónica roja en el estadio, que se muestra en el fondo de la ventana del comentarista.
Mencionada: NO
---
Resumen: Total de apariciones: 1
[FIN ANÁLISIS]


In [23]:
def pasar_json(respuesta_gemini: str):
    """
    Limpia la respuesta de texto de Gemini y la convierte en un objeto JSON.
    
    Args:
        respuesta_gemini: La cadena de texto cruda devuelta por `respuesta.text`.

    Returns:
        Un diccionario o lista de Python, o None si ocurre un error.
    """
    try:
        # 1. Limpia el string de los marcadores de Markdown y espacios en blanco
        # Usamos removeprefix y removesuffix que son más seguros que replace
        json_limpio_str = respuesta_gemini.strip().removeprefix('```json').removesuffix('```').strip()
        
        # 2. Convierte el string limpio en un objeto JSON (diccionario/lista)
        return json.loads(json_limpio_str)
        
    except json.JSONDecodeError:
        print("Error: La respuesta de Gemini no pudo ser decodificada como JSON válido.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None

# **Pruebas**

### **Primer tiempo**

##### Pruebas con mi sistem prompt 

In [27]:
# Marcamos el inicio del programa
start_time = time.time()


# Obtenemos la lista de videos
uris = obtener_nombres_de_archivos(nombre_bucket=BUCKET_NAME, nombre_carpeta=VIDEO_FOLDER_PATH)

uris

data = [] # Aquí vamos a guardar los JSON


for i in range(233):
    print(f'Vamos en {uris[i]}')
    data.append( ['gs://vboxiooof/Videos/Videos_Segmentados/'+uris[i], pasar_json(reporteGemini(uris[i]))] ) # Guardamos tuplas (uri, info)


end_time = time.time()


print(f'El proceso duró: {end_time - start_time}')

Vamos en mexicosta_segment_000.mkv
Vamos en mexicosta_segment_001.mkv
Vamos en mexicosta_segment_002.mkv
Vamos en mexicosta_segment_003.mkv
Vamos en mexicosta_segment_004.mkv
Vamos en mexicosta_segment_005.mkv
Vamos en mexicosta_segment_006.mkv
Vamos en mexicosta_segment_007.mkv
Vamos en mexicosta_segment_008.mkv
Vamos en mexicosta_segment_009.mkv
Vamos en mexicosta_segment_010.mkv
Vamos en mexicosta_segment_011.mkv
Vamos en mexicosta_segment_012.mkv
Vamos en mexicosta_segment_013.mkv
Vamos en mexicosta_segment_014.mkv
Vamos en mexicosta_segment_015.mkv
Vamos en mexicosta_segment_016.mkv
Vamos en mexicosta_segment_017.mkv
Vamos en mexicosta_segment_018.mkv
Vamos en mexicosta_segment_019.mkv
Vamos en mexicosta_segment_020.mkv
Vamos en mexicosta_segment_021.mkv
Vamos en mexicosta_segment_022.mkv
Vamos en mexicosta_segment_023.mkv
Vamos en mexicosta_segment_024.mkv
Vamos en mexicosta_segment_025.mkv
Vamos en mexicosta_segment_026.mkv
Vamos en mexicosta_segment_027.mkv
Vamos en mexicosta_s

In [36]:
print(reporteGemini('mexicosta_segment_052.mkv', 'Caliente'))

```json
{
  "apariciones": [
    {
      "marca": "Caliente",
      "categoria": "Banner",
      "inicio": "00:00",
      "fin": "00:04"
    },
    {
      "marca": "Caliente",
      "categoria": "Valla",
      "inicio": "00:00",
      "fin": "00:16"
    },
    {
      "marca": "Caliente",
      "categoria": "Marcador",
      "inicio": "00:07",
      "fin": "00:08"
    },
    {
      "marca": "Caliente",
      "categoria": "Marcador",
      "inicio": "00:09",
      "fin": "00:11"
    },
    {
      "marca": "Caliente",
      "categoria": "Marcador",
      "inicio": "00:13",
      "fin": "00:15"
    }
  ]
}
```


In [33]:
print(reporteAlain('mexicosta_segment_052.mkv', 'Caliente'))

[INICIO ANÁLISIS]
Video: gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_052.mkv
Marca buscada: Caliente
---
Detecciones:
[Detección 1]
Tiempo: 00:00:00 - 00:00:04
Tipo: Banner
Tamaño: Grande (>30%)
Ubicación: Inferior y lateral izquierdo
Contexto: Un banner rojo en la parte inferior muestra el logo de Caliente.mx, el texto "CASA DE APUESTAS OFICIAL" y momios para tiros de esquina. Simultáneamente, un banner vertical en el lado izquierdo muestra el logo y una app en un teléfono.
Mencionada: NO
---
[Detección 2]
Tiempo: 00:00:00 - 00:00:31
Tipo: Vallas electronicas
Tamaño: Pequeño (<10%)
Ubicación: Fondo, detrás de la portería
Contexto: Las vallas publicitarias electrónicas que rodean el campo de juego muestran repetidamente el logo y el texto "caliente.mx".
Mencionada: NO
---
[Detección 3]
Tiempo: 00:00:22 - 00:00:31
Tipo: Marcador
Tamaño: Pequeño (<10%)
Ubicación: Esquina superior izquierda
Contexto: El logo de "Caliente.mx" aparece en un recuadro rojo justo al lado del mar

In [28]:
data

[['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_000.mkv',
  {'apariciones': [{'marca': 'Caliente',
     'categoria': 'Valla',
     'inicio': '00:11',
     'fin': '00:15'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_001.mkv',
  {'apariciones': [{'marca': 'Caliente',
     'categoria': 'Valla',
     'inicio': '00:01',
     'fin': '00:06'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_002.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_003.mkv',
  {'apariciones': [{'marca': 'Caliente',
     'categoria': 'Anuncio',
     'inicio': '00:00',
     'fin': '00:15'}]}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_004.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_005.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_006.mkv',
  {'apariciones': []}],
 ['gs://vboxiooof/Videos/Videos_Segmentados/mex

* 000 erroneo (no sale nada)
* 003 erroneo (no sale nada)
* 019 erroneo (no sale nada)
* 021 erroneo (no es spot, es virtual)
* 022 erroneo (no es ni spot, ni banner, es virtual)
* 023 erroneo (no es spot, es virtual)
* 027 erroneo (no sale nada)
* 028 erroneo (no sale nada)
* 001 erroneo (no sale nada)
* 030 erroneo (no sale nada)
* 031 erroneo (no sale nada)
* 032 erroneo (no sale nada)
* 037 erroneo (no sale nada)
* 043 erroneo (no es anuncio, es virtual)
* 044 erroneo (no es anuncio, es virtual)
* 045 erroneo (no es anuncio, es virtual, y los tiempos van de 0 a 2, no de 0 a 15)
* 046 erroneo (no sale nada)
* 047 erroneo (no sale nada)
* 049 erroneo (no sale nada)
* 050 erroneo (no sale nada)















In [44]:
print(reporteAlain('mexicosta_segment_001.mkv', 'Caliente.mx'))

[INICIO ANÁLISIS]
Video: gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_001.mkv
Marca buscada: Caliente.mx
---
Detecciones:
[Detección 1]
Tiempo: 00:00:02 - 00:00:07
Tipo: Vallas electronicas
Tamaño: Mediano
Ubicación: Centro, fondo del campo
Contexto: La marca aparece de forma prominente en las vallas electrónicas de color rojo que rodean el campo de juego, durante la repetición de una jugada de gol.
Mencionada: NO
---
Resumen: Total de apariciones: 1
[FIN ANÁLISIS]


In [42]:
print(reporteGemini('mexicosta_segment_001.mkv', 'Caliente'))

```json
{
  "apariciones": [
    {
      "marca": "Caliente",
      "categoria": "Valla",
      "inicio": "00:02",
      "fin": "00:06"
    }
  ]
}
```



##### Pruebas con el sistem prompt de Alain

In [ ]:
# Marcamos el inicio del programa
start_time = time.time()


# Obtenemos la lista de videos
uris = obtener_nombres_de_archivos(nombre_bucket=BUCKET_NAME, nombre_carpeta=VIDEO_FOLDER_PATH)

uris

data = [] # Aquí vamos a guardar los JSON


for i in range(233):
    print(reporteAlain(uris[i], 'Caliente'))
    print('-'*100)


end_time = time.time()


print(f'El proceso duró: {end_time - start_time}')

[INICIO ANÁLISIS]
Video: gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_000.mkv
Marca buscada: Caliente
---
Detecciones:
[Detección 1]
Tiempo: 00:00:10 - 00:00:15
Tipo: Valla electronica
Tamaño: pequeño
Ubicación: esquina inferior derecha (en el fondo de la ventana del comentarista)
Contexto: El logo de "Caliente.mx" es visible en una valla electrónica roja en el estadio, que se muestra en el fondo de la ventana del comentarista.
Mencionada: NO
---
Resumen: Total de apariciones: 1
[FIN ANÁLISIS]
----------------------------------------------------------------------------------------------------
[INICIO ANÁLISIS]
Video: gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_001.mkv
Marca buscada: Caliente
---
Detecciones:
[Detección 1]
Tiempo: 00:00:02 - 00:00:07
Tipo: Vallas electronicas
Tamaño: Mediano
Ubicación: Perímetro del campo
Contexto: La marca "caliente.mx" se muestra en las vallas electrónicas de color rojo que rodean el campo, tanto en la línea de banda como 